In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import os
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

In [5]:
TRAIN_PATH = 'June to Sep 2018_SC.csv'

def read_large_file(path = TRAIN_PATH, chunksize = 1000, col= None,
                    filename= 'ticket.feather',format_type = '%Y-%m-%d %H:%M'):

    df_list = []
    for df_chunk in tqdm(pd.read_csv(path,chunksize= chunksize)):


        try:

            for c in col:

                df_chunk[c] = df_chunk[c].str.slice(0, 16)

                df_chunk[c] = pd.to_datetime(df_chunk[c], format=format_type)

                df_list.append(df_chunk)

        except:

            print("Operation failed to perform")

            return None

    train_df = pd.concat(df_list)

    del df_list

    train_df.to_feather(os.path.join(TRAIN_PATH + filename))

    return train_df


df= read_large_file(col=['date'])

0it [00:00, ?it/s]

Operation failed to perform


In [ ]:
df = pd.read_csv()

In [ ]:
df_ageband1 = df[df.age.between(10,20)]

In [ ]:
top20_title = df_ageband1.groupby('title_id')['impressions'].sum().sort_values(ascending=False).head(20)
top20_frame = top20_title.to_frame()
# top20_frame

In [ ]:
df_ageband1_sorted = df_ageband1.groupby(['date','title_id'])['impressions'].sum().to_frame().sort_values(by='impressions',ascending=False)
age_band1_final = df_ageband1_sorted.unstack(level='title_id')
age_band1_final.columns = age_band1_final.columns.droplevel()

age_band1_final = age_band1_final[top20_frame.index]

change_colum = ['t985.0','t1533.0','t979.0','t41.0','t978.0','t1075.0','t39.0','t12.0','t1599.0','t1341.0','t53.0','t1006.0','t922.0','t1048.0',
                't680.0','t445.0','t1540.0','t813.0','t52.0','t55.0']

In [ ]:
age_band1_final.columns = change_colum

In [ ]:
age_band1_final =  age_band1_final.resample('W').mean()

In [3]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mean_absolute_percentage_error_by_row(y_true, y_pred):
    return np.abs((y_true - y_pred) / y_true) * 100

In [ ]:
mean0 = []
mean1 = []
mean2 = []
std0 = []
std1 = []
std2 = []

for titleID in change_colum:
    mean0.append(age_band1_final.loc['2018-01-01':'2018-01-31',titleID].mean())
    mean1.append(age_band1_final.loc['2018-02-01':'2018-02-28',titleID].mean())
    mean2.append(age_band1_final.loc['2018-03-01':'2018-03-31',titleID].mean())
    std0.append(age_band1_final.loc['2018-01-01':'2018-01-31',titleID].std())
    std1.append(age_band1_final.loc['2018-02-01':'2018-02-28',titleID].std())
    std2.append(age_band1_final.loc['2018-03-01':'2018-03-31',titleID].std())
    
month0 = age_band1_final['2018-01-01':'2018-01-31']
month1 = age_band1_final['2018-02-01':'2018-02-28']
month2 = age_band1_final['2018-03-01':'2018-03-31']


for k,i in enumerate(change_colum):
    month0.loc[month0[i]>mean0[k]+std0[k],i] = mean0[k]+std0[k]
    month1.loc[month1[i]>mean1[k]+std1[k],i] = mean1[k]+std1[k]
    month2.loc[month2[i]>mean2[k]+std2[k],i] = mean2[k]+std2[k]
    
    month0.loc[month0[i]<mean0[k]-std0[k],i] = mean0[k]-std0[k]
    month1.loc[month1[i]<mean1[k]-std1[k],i] = mean1[k]-std1[k]
    month2.loc[month2[i]<mean2[k]-std2[k],i] = mean2[k]-std2[k]

In [ ]:
outlier_free_df = pd.concat([month0,month1,month2])
outlier_free_df.fillna(0,inplace=True)

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing
ets = outlier_free_df
test_pred = []
mape = []
smape_error = []
i=0
for j in change_colum:
    
    train_ets = outlier_free_df.loc['2018-01-01':'2018-03-15',j].to_frame()
    test_ets = outlier_free_df.loc['2018-03-16':'2018-03-31',j].to_frame()
    
    ETS = ExponentialSmoothing(train_ets,trend='add',damped=True).fit(smoothing_level=0.2,smoothing_slope=0.1,optimized=True)
    #ETS.forecast(len(test_ets)-1)
    predicted_val = ETS.predict(start='2018-03-16',end='2018-03-31')
    test_pred.append(predicted_val)
    pred = pd.DataFrame(predicted_val,index=test_ets.index,columns=[j])
    mape.append(mean_absolute_percentage_error(test_ets[j],pred[j]))
    smape_error.append(smape(test_ets[j].tolist(),test_pred[i]))
    i = i+1
pd.DataFrame(smape_error).T.to_excel('ets_smape.xlsx')  
#np.mean(mape)
# pd.DataFrame(mape).T.to_excel('ets_mape.xlsx')
#pd.DataFrame(test_pred).T.to_excel('ets_pred.xlsx')


In [ ]:
forecast_pred = []
for j in change_colum:
    ETS = ExponentialSmoothing(outlier_free_df[j],trend='add',damped=True).fit(smoothing_level=0.2,smoothing_slope=0.1,optimized=True)
    predicted_val = ETS.predict(start='2018-04-01',end='2018-04-15')
    forecast_pred.append(predicted_val)
forecast_df = pd.DataFrame(forecast_pred)
forecast_t = forecast_df.T
# forecast_t.to_excel('forecast_ets.xlsx')
# forecast_t

In [ ]:
from pyramid.arima import auto_arima
test_pred = []
mape = []
i=0
smape_error = []
for j in change_colum:
    train_df = outlier_free_df.loc['2018-02-01':'2018-03-15',j].to_frame()
    test_df  = outlier_free_df.loc['2018-03-16':'2018-03-31',j].to_frame()
    stepwise_model = auto_arima(train_df, start_p=0, start_q=0,
                       max_p=2, max_q=2, m=7,
                       start_P=0, seasonal=True,
                       d=1, D=1, trace=False,
                       error_action='ignore',
                       suppress_warnings=True, 
                       stepwise=True,stationary=True)
    stepwise_model.fit(test_df)
    predicted_val = stepwise_model.predict(n_periods=2)
    test_pred.append(predicted_val)
    pred = pd.DataFrame(predicted_val,index=test_df.index,columns=[j])
    mape.append(mean_absolute_percentage_error(test_df[j],pred[j]))
    smape_error.append(smape(test_df[j].tolist(),test_pred[i]))
    #i = i+1
#pd.DataFrame(smape_error).T.to_excel('arima_smape.xlsx') 
# smape_error


In [ ]:
test_forecast = pd.DataFrame(test_pred).T
test_forecast.to_excel('test_forcast.xlsx')

In [ ]:
forecast_pred = []
for j in change_colum:
    stepwise_model = auto_arima(outlier_free_df[j], start_p=0, start_q=0,
                       max_p=2, max_q=2, m=1,
                       start_P=0, seasonal=True,
                       d=1, D=1, trace=False,
                       error_action='ignore',
                       suppress_warnings=True, 
                       stepwise=True,stationary=True)
    stepwise_model.fit(outlier_free_df[j])
    predicted_val = stepwise_model.predict(n_periods=16)
    forecast_pred.append(predicted_val)
forecast_df = pd.DataFrame(forecast_pred)
forecast_t = forecast_df.T
forecast_t.to_excel('forecast_data.xlsx')


In [263]:
df_ageband2 = df[df.age.between(20,35)]
other = df_ageband2[df_ageband2.gender == 0]
male = df_ageband2[df_ageband2.gender == 1]
female = df_ageband2[df_ageband2.gender == 2]

In [266]:
# top20_male_title = male.groupby('title_id')['impressions'].sum().sort_values(ascending=False).head(20)
# top20_male_frame = top20_male_title.to_frame()
# top20_male_frame.index.to_frame().T.to_excel('male_top20.xlsx')

# top20_female_title = female.groupby('title_id')['impressions'].sum().sort_values(ascending=False).head(20)
# top20_female_frame = top20_female_title.to_frame()
# # # top20_female_frame.index.to_frame().T.to_excel('female_top20.xlsx')

top20_other_title = other.groupby('title_id')['impressions'].sum().sort_values(ascending=False).head(20)
top20_other_frame = top20_other_title.to_frame()
# # top20_other_frame.index.to_frame().T.to_excel('other_top20.xlsx')

In [269]:
# df_male_sorted = male.groupby(['date','title_id'])['impressions'].sum().to_frame().sort_values(by='impressions',ascending=False)
# df_male_final = df_male_sorted.unstack(level='title_id')
# df_male_final.columns = df_male_final.columns.droplevel()
# df_male_final.fillna(0,inplace=True)
# df_male_final = df_male_final[top20_male_frame.index]
# df_male_final
# df_male_final.to_excel('maleagewise_titleid20-35orignal.xlsx')


# df_female_sorted = female.groupby(['date','title_id'])['impressions'].sum().to_frame().sort_values(by='impressions',ascending=False)
# df_female_final = df_female_sorted.unstack(level='title_id')
# df_female_final.columns = df_female_final.columns.droplevel()
# df_female_final.fillna(0,inplace=True)
# df_female_final = df_female_final[top20_female_frame.index]

# # # df_female_final.to_excel('femaleagewise_titleid20-35orignal.xlsx')


df_other_sorted = other.groupby(['date','title_id'])['impressions'].sum().to_frame().sort_values(by='impressions',ascending=False)
df_other_final = df_other_sorted.unstack(level='title_id')
df_other_final.columns = df_other_final.columns.droplevel()
df_other_final.fillna(0,inplace=True)
df_other_final = df_other_final[top20_other_frame.index]

# # df_other_final.to_excel('otheragwise_titleid20-35orignal.xlsx')

In [270]:
#df_male_final.columns = df_male_final.columns.map(str)
#df_female_final.columns = df_female_final.columns.map(str)
df_other_final.columns = df_other_final.columns.map(str)

In [271]:

def outlier_correction_agegroup(df):
    mean0 = []
    mean1 = []
    mean2 = []
    std0 = []
    std1 = []
    std2 = []
    for titleID in df.columns:
        mean0.append(df.loc['2018-01-01':'2018-01-31',titleID].mean())
        mean1.append(df.loc['2018-02-01':'2018-02-28',titleID].mean())
        mean2.append(df.loc['2018-03-01':'2018-03-31',titleID].mean())
        std0.append(df.loc['2018-01-01':'2018-01-31',titleID].std())
        std1.append(df.loc['2018-02-01':'2018-02-28',titleID].std())
        std2.append(df.loc['2018-03-01':'2018-03-31',titleID].std())

    month0 = df['2018-01-01':'2018-01-31']
    month1 = df['2018-02-01':'2018-02-28']
    month2 = df['2018-03-01':'2018-03-31']
    
    for k,i in enumerate(df.columns):
        
        month0.loc[month0[i]>mean0[k]+std0[k],i] = mean0[k]+std0[k]
        month1.loc[month1[i]>mean1[k]+std1[k],i] = mean1[k]+std1[k]
        month2.loc[month2[i]>mean2[k]+std2[k],i] = mean2[k]+std2[k]

        month0.loc[month0[i]<mean0[k]-std0[k],i] = mean0[k]-std0[k]
        month1.loc[month1[i]<mean1[k]-std1[k],i] = mean1[k]-std1[k]
        month2.loc[month2[i]<mean2[k]-std2[k],i] = mean2[k]-std2[k]

    return [month0,month1,month2]
#month0,month1,month2 = outlier_correction_agegroup(df_male_final)
#month4,month5,month6 = outlier_correction_agegroup(df_female_final)
month7,month8,month9 = outlier_correction_agegroup(df_other_final)

In [273]:
# outlier_free_df_male = pd.concat([month0,month1,month2])
# outlier_free_df_male.fillna(0,inplace=True)
#outlier_free_df_male.to_excel('male_outlierfree.xlsx')


# outlier_free_df_female = pd.concat([month4,month5,month6])
# outlier_free_df_female.fillna(0,inplace=True)
# # #outlier_free_df_female.to_excel('female_outlierfree.xlsx')

outlier_free_df_other = pd.concat([month7,month8,month9])
outlier_free_df_other.fillna(0,inplace=True)
# #outlier_free_df_other.to_excel('other_outlierfree.xlsx')

In [274]:
outlier_free_df_other.columns

Index(['985.0', '1533.0', '1075.0', '53.0', '41.0', '1006.0', '1341.0',
       '978.0', '979.0', '922.0', '445.0', '680.0', '39.0', '1.0', '1029.0',
       '1048.0', '24.0', '793.0', '52.0', '1080.0'],
      dtype='object', name='title_id')

In [275]:

def sarfunc(df,title_id):
    order_final = []
    smape_final = []
    final_data = []
    #title_id = '985.0'
    train_df = df.loc['2018-01-01':'2018-03-15',title_id].to_frame()
    test_df  = df.loc['2018-03-16':'2018-03-31',title_id].to_frame()
    for p in range(0,3):
        for q in range(0,3):
            for d in range(0,3):
                for P in range(0,3):
                    for Q in range(0,3):
                        for D in range(0,3):
                            order = (p,d,q)
                            Order = (P,D,Q,7)
                            mod =  sm.tsa.statespace.SARIMAX(train_df, trend='n', order=order, 
                                                             seasonal_order=Order, enforce_stationarity=False,
                                                             enforce_invertibility=False)
                            results = mod.fit(method='powell')
                            spred = results.predict(start='2018-03-16',end='2018-03-31')
                            pred = spred.to_frame()
                            smapp = smape(test_df[title_id].tolist(),spred)
                            if smapp < 20:
    #                             print("-"*40)
                                smape_final.append(smapp)
                                order_final.append([order,Order])
                                final_data.append(pred[0].values.tolist())
    #                             plt.plot(subplots=True)
    #                             plt.plot(pd.concat([train_df, test_df],axis=1),label='orignal')
    #                             plt.plot(pd.DataFrame(results.predict(start='2018-03-16',end='2018-03-31')),
    #                                      label='pred')
    #                             plt.legend()
    # #                             plt.show()
    #                             print('order=>{}-{}-{}-{}-{}-{}'.format(p,d,q,P,D,Q),
    #                                   mean_absolute_percentage_error(test_df[title_id],pred[0]),smapp)
    #                             print(pred)
    #                             print("-"*40)
    smape_10 = pd.DataFrame(smape_final,columns=['smape']).sort_values(by='smape',ascending=True).head(10)
    order_10 = pd.DataFrame(order_final,columns=['order1','order2'])
    data_10 = pd.DataFrame(final_data)
    order_10 = order_10.loc[smape_10.index]
    data_10 = data_10.loc[smape_10.index]
    final_df = pd.concat([data_10,smape_10,order_10],axis=1)
    return final_df

In [307]:
# [ 985.0, 1533.0, 1075.0,   53.0,   41.0, 1006.0, 1341.0,  978.0,
#                979.0,  922.0,  445.0,  680.0,   39.0,    1.0, 1029.0, 1048.0,
#                 24.0,  793.0,   52.0, 1080.0]
column_df = sarfunc(outlier_free_df_other,'922.0')

Optimization terminated successfully.
         Current function value: 10.422463
         Iterations: 1
         Function evaluations: 11
Optimization terminated successfully.
         Current function value: 8.748696
         Iterations: 2
         Function evaluations: 41
Optimization terminated successfully.
         Current function value: 8.314488
         Iterations: 2
         Function evaluations: 37
Optimization terminated successfully.
         Current function value: 9.190631
         Iterations: 2
         Function evaluations: 48
Optimization terminated successfully.
         Current function value: 7.736469
         Iterations: 2
         Function evaluations: 56
Optimization terminated successfully.
         Current function value: 6.972714
         Iterations: 3
         Function evaluations: 80
Optimization terminated successfully.
         Current function value: 7.976305
         Iterations: 3
         Function evaluations: 118
Optimization terminated successfully.
 

Optimization terminated successfully.
         Current function value: 7.312406
         Iterations: 3
         Function evaluations: 99
Optimization terminated successfully.
         Current function value: 6.492318
         Iterations: 5
         Function evaluations: 194
Optimization terminated successfully.
         Current function value: 5.732943
         Iterations: 5
         Function evaluations: 220
Optimization terminated successfully.
         Current function value: 8.294161
         Iterations: 1
         Function evaluations: 59
Optimization terminated successfully.
         Current function value: 7.575537
         Iterations: 1
         Function evaluations: 54
Optimization terminated successfully.
         Current function value: 6.969009
         Iterations: 1
         Function evaluations: 53
Optimization terminated successfully.
         Current function value: 8.165497
         Iterations: 3
         Function evaluations: 111
Optimization terminated successfully.


Optimization terminated successfully.
         Current function value: 7.050267
         Iterations: 4
         Function evaluations: 182
Optimization terminated successfully.
         Current function value: 6.372225
         Iterations: 3
         Function evaluations: 155
Optimization terminated successfully.
         Current function value: 6.994257
         Iterations: 3
         Function evaluations: 168
Optimization terminated successfully.
         Current function value: 6.214470
         Iterations: 5
         Function evaluations: 296
Optimization terminated successfully.
         Current function value: 5.469787
         Iterations: 5
         Function evaluations: 306
Optimization terminated successfully.
         Current function value: 7.226511
         Iterations: 2
         Function evaluations: 80
Optimization terminated successfully.
         Current function value: 6.512682
         Iterations: 2
         Function evaluations: 82
Optimization terminated successfully

Optimization terminated successfully.
         Current function value: 6.668834
         Iterations: 3
         Function evaluations: 169
Optimization terminated successfully.
         Current function value: 5.950471
         Iterations: 4
         Function evaluations: 239
Optimization terminated successfully.
         Current function value: 7.527062
         Iterations: 3
         Function evaluations: 222
Optimization terminated successfully.
         Current function value: 6.670060
         Iterations: 3
         Function evaluations: 200
Optimization terminated successfully.
         Current function value: 5.830014
         Iterations: 3
         Function evaluations: 222
Optimization terminated successfully.
         Current function value: 7.161305
         Iterations: 4
         Function evaluations: 324
Optimization terminated successfully.
         Current function value: 6.302226
         Iterations: 3
         Function evaluations: 239
Optimization terminated successful

Optimization terminated successfully.
         Current function value: 5.979911
         Iterations: 5
         Function evaluations: 405
Optimization terminated successfully.
         Current function value: 5.270674
         Iterations: 5
         Function evaluations: 432
Optimization terminated successfully.
         Current function value: 8.979806
         Iterations: 1
         Function evaluations: 45
Optimization terminated successfully.
         Current function value: 8.519909
         Iterations: 1
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: 8.134410
         Iterations: 1
         Function evaluations: 70
Optimization terminated successfully.
         Current function value: 8.075407
         Iterations: 3
         Function evaluations: 94
Optimization terminated successfully.
         Current function value: 7.367334
         Iterations: 3
         Function evaluations: 102
Optimization terminated successfully.


...

Optimization terminated successfully.
         Current function value: 5.485970
         Iterations: 6
         Function evaluations: 548
Optimization terminated successfully.
         Current function value: 6.772794
         Iterations: 4
         Function evaluations: 369
Optimization terminated successfully.
         Current function value: 5.974694
         Iterations: 4
         Function evaluations: 371
Optimization terminated successfully.
         Current function value: 5.213729
         Iterations: 6
         Function evaluations: 591
Optimization terminated successfully.
         Current function value: 8.856161
         Iterations: 1
         Function evaluations: 64
Optimization terminated successfully.
         Current function value: 8.392995
         Iterations: 1
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: 7.999473
         Iterations: 1
         Function evaluations: 79
Optimization terminated successfully.

Optimization terminated successfully.
         Current function value: 7.936378
         Iterations: 1
         Function evaluations: 68
Optimization terminated successfully.
         Current function value: 7.975398
         Iterations: 2
         Function evaluations: 81
Optimization terminated successfully.
         Current function value: 7.243687
         Iterations: 3
         Function evaluations: 142
Optimization terminated successfully.
         Current function value: 6.725144
         Iterations: 3
         Function evaluations: 134
Optimization terminated successfully.
         Current function value: 7.128976
         Iterations: 3
         Function evaluations: 153
Optimization terminated successfully.
         Current function value: 6.338721
         Iterations: 4
         Function evaluations: 251
Optimization terminated successfully.
         Current function value: 5.600421
         Iterations: 6
         Function evaluations: 359
Optimization terminated successfully

Optimization terminated successfully.
         Current function value: 5.514807
         Iterations: 6
         Function evaluations: 416
Optimization terminated successfully.
         Current function value: 7.810206
         Iterations: 2
         Function evaluations: 99
Optimization terminated successfully.
         Current function value: 7.168978
         Iterations: 2
         Function evaluations: 108
Optimization terminated successfully.
         Current function value: 6.595540
         Iterations: 2
         Function evaluations: 111
Optimization terminated successfully.
         Current function value: 7.810292
         Iterations: 2
         Function evaluations: 122
Optimization terminated successfully.
         Current function value: 7.043447
         Iterations: 4
         Function evaluations: 258
Optimization terminated successfully.
         Current function value: 6.369388
         Iterations: 3
         Function evaluations: 212
Optimization terminated successfull

Optimization terminated successfully.
         Current function value: 6.332784
         Iterations: 6
         Function evaluations: 491
Optimization terminated successfully.
         Current function value: 6.989038
         Iterations: 14
         Function evaluations: 1213
Optimization terminated successfully.
         Current function value: 6.185747
         Iterations: 4
         Function evaluations: 373
Optimization terminated successfully.
         Current function value: 5.449734
         Iterations: 7
         Function evaluations: 663
Optimization terminated successfully.
         Current function value: 7.102205
         Iterations: 6
         Function evaluations: 448
Optimization terminated successfully.
         Current function value: 6.358950
         Iterations: 4
         Function evaluations: 303
Optimization terminated successfully.
         Current function value: 5.639397
         Iterations: 5
         Function evaluations: 397
Optimization terminated successf

Optimization terminated successfully.
         Current function value: 5.429142
         Iterations: 6
         Function evaluations: 520
Optimization terminated successfully.
         Current function value: 6.886838
         Iterations: 3
         Function evaluations: 259
Optimization terminated successfully.
         Current function value: 6.152240
         Iterations: 4
         Function evaluations: 387
Optimization terminated successfully.
         Current function value: 5.356244
         Iterations: 7
         Function evaluations: 700
Optimization terminated successfully.
         Current function value: 6.768239
         Iterations: 4
         Function evaluations: 409
Optimization terminated successfully.
         Current function value: 5.971310
         Iterations: 4
         Function evaluations: 417
Optimization terminated successfully.
         Current function value: 5.220849
         Iterations: 6
         Function evaluations: 624


In [308]:
#column_df
column_dfT = column_df.T
#column_dfT.to_excel('predicted.xlsx')
column_dfT

243           12            17            49            143  \
0              12563       12859.5       12620.3       12070.9       12135.9   
1            12956.6       12691.5       12416.5       11877.2       11962.3   
2            12882.2       13089.6       12495.6       11893.8       11991.4   
3              10653       13023.1       13093.9       12789.9       12875.9   
4            12247.5       13295.2       17334.8       17376.8       17454.2   
5            13905.9       13758.3       15135.1       14990.8       15069.4   
6            14434.1       14103.6         15356       15372.8       15456.1   
7            13952.8       14126.1       14629.9       14328.5       14431.7   
8            14168.3       13958.1       13947.1       13660.4       13772.3   
9            13845.2       14356.2       14212.3       13846.8       13974.8   
10           11332.2       14289.7       13183.9       13196.2       13313.7   
11           13103.9       14561.8       14681.9       15136.7       15250.4   
12           14664.6       15024.9       14045.6       14261.8       14373.2   
13           15240.7       15370.2       16477.8         16885         16997   
14           14724.4       15392.7         16137       15866.1       16035.7   
15             14900       15224.7       15816.5       15567.5       15757.4   
smape        10.2445       10.3103       10.3927       10.4107        10.464   
order1     (2, 0, 0)     (0, 1, 0)     (0, 1, 0)     (0, 1, 1)     (1, 1, 0)   
order2  (0, 2, 2, 7)  (0, 1, 1, 7)  (1, 1, 0, 7)  (1, 2, 1, 7)  (1, 2, 1, 7)   

                 197           124           176           140           339  
0            13429.6       12553.8         12680       12251.8       12221.7  
1            13698.1       12915.1         12921       12072.5       11826.4  
2            13351.8         12816       13114.1       12150.2       11930.1  
3            11566.6       10624.2       13312.9       12746.6       12885.5  
4            12971.6       12187.1         13511       16983.9       17492.8  
5            13818.2       13803.5       13709.3       14786.2       15050.1  
6            14471.6       14316.6       13907.5       15009.9       15422.2  
7            14268.6       13845.5       14105.7       14194.4       14430.4  
8              14454       14025.2       14303.9         13518       13705.9  
9              14210       13685.7       14502.1       13782.6       13926.8  
10             12200       11252.6       14700.3       12755.4       13269.2  
11           13840.5       12994.6       14898.5       14255.1       15207.9  
12           14777.5       14481.2       15096.7       13617.8       14303.6  
13           15479.4       15051.9       15294.9       16048.5       16946.1  
14           15232.9       14559.2       15493.1       15407.1       16070.6  
15           15483.2       14686.5       15691.3       15106.4       15605.6  
smape        10.4761       10.5787       10.5953       10.6056       10.6178  
order1     (1, 0, 2)     (1, 0, 0)     (1, 2, 1)     (1, 1, 0)     (2, 1, 2)  
order2  (0, 2, 2, 7)  (0, 2, 2, 7)  (0, 0, 0, 7)  (1, 1, 0, 7)  (1, 2, 1, 7)

In [309]:
order1 = column_dfT.loc['order1']
order1 = order1.reset_index(drop=True)
order2 = column_dfT.loc['order2']
order2 = order2.reset_index(drop=True)

In [279]:
def sarfore(df,titleId):
    cl = pd.DataFrame(index=['2018-04-01','2018-04-02','2018-04-03','2018-04-04','2018-04-05','2018-04-06',
                            '2018-04-07','2018-04-08','2018-04-09','2018-04-10','2018-04-11','2018-04-12',
                            '2018-04-13','2018-04-14','2018-04-15'])
    for i in range(len(order1)):
        mod =  sm.tsa.statespace.SARIMAX(df[titleId], trend='n', order=order1[i], seasonal_order=order2[i], enforce_stationarity=False,
                     enforce_invertibility=False)
        results = mod.fit()
        p = pd.DataFrame(results.predict(start='2018-04-01',end='2018-04-15'))
        cl = pd.concat([cl,p],axis=1)
    return cl

In [310]:
# ['985.0', '1533.0', '41.0', '978.0', '53.0', '39.0', '1341.0', '979.0',
#        '1599.0', '1075.0', '1392.0', '1048.0', '1006.0', '24.0', '445.0',
#        '12.0', '33.0', '52.0', '793.0', '1.0']
sarfore(outlier_free_df_other,'922.0')
#fore.to_excel('femafore.xlsx')

/home/hadoopUser/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/home/hadoopUser/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0             0             0             0  \
2018-04-01  10087.371560   9496.798902   8917.178602   9145.048253   
2018-04-02   8556.215376   9001.447087   6789.529230   6974.889026   
2018-04-03  11805.781508   9630.767479   9858.716978  10329.715117   
2018-04-04  13115.332956  10115.821773  10678.981345  11190.829958   
2018-04-05  14340.670708  10533.886318  11384.114324  11947.946526   
2018-04-06  14001.297416  10392.778195  11095.971032  11500.902628   
2018-04-07  12909.076914   9930.383929   9659.809774   9746.776676   
2018-04-08  12925.870493  10249.182831   9587.954492   9664.077329   
2018-04-09  10821.495430   9753.831016   7169.460539   7194.242680   
2018-04-10  14056.102922  10383.151408   9326.361874   9725.831512   
2018-04-11  14760.389639  10868.205702   9421.206319   9880.428353   
2018-04-12  15890.685456  11286.270248  10267.585704  10791.812603   
2018-04-13  15328.076107  11145.162124   9203.538076   9532.628720   
2018-04-14  13580.383325  10682.767859   5031.042484   4981.929056   
2018-04-15  13501.456764  11001.566760   4822.280517   4861.597782   

                       0             0             0             0  \
2018-04-01   9113.754502  10107.076163  10119.459364   9829.331894   
2018-04-02   6923.872838   9019.208574   9064.507640   9789.648661   
2018-04-03  10273.240669  11690.819613  12776.572881   9831.390204   
2018-04-04  11133.738109  12717.864861  13078.756415   9863.537175   
2018-04-05  11890.894892  13925.150970  14611.643124   9896.814708   
2018-04-06  11442.979224  13638.872502  14242.448043   9929.959024   
2018-04-07   9685.438660  12418.813054  12021.477491   9963.119036   
2018-04-08   9599.974237  12459.827859  12306.750676   9996.277200   
2018-04-09   7118.280514  10680.622829  10702.640528  10029.435581   
2018-04-10   9644.499128  13664.912012  14612.439321  10062.593936   
2018-04-11   9799.984060  14084.880169  14292.775209  10095.752295   
2018-04-12  10711.138181  15267.211895  15911.820810  10128.910653   
2018-04-13   9452.463953  14811.763265  15344.839988  10162.069011   
2018-04-14   4903.290984  12855.635182  12298.924959  10195.227369   
2018-04-15   4765.599710  12861.961625  12611.124467  10228.385727   

                       0             0  
2018-04-01   9228.617420   9633.061273  
2018-04-02   7083.489802   8586.691083  
2018-04-03  10151.881194  12624.284007  
2018-04-04  10970.729474  13954.471860  
2018-04-05  11676.131144  15050.059713  
2018-04-06  11386.526706  14828.164707  
2018-04-07   9945.213047  13194.852504  
2018-04-08   9959.207978  13378.596272  
2018-04-09   7536.278335  11385.975832  
2018-04-10   9694.204042  14203.735462  
2018-04-11   9789.646588  14517.631771  
2018-04-12  10635.907572  15541.219100  
2018-04-13   9572.514458  14347.537231  
2018-04-14   5402.327248   9814.630955  
2018-04-15   5442.819117  10129.010597

In [57]:
from pyramid.arima import auto_arima

def prediction_mape(df):
    test_pred = []
    mape = []
    i=0
    smape_error = []
    for j in df.columns:
        train_df = df.loc['2018-01-01':'2018-03-15',j].to_frame()
        test_df  = df.loc['2018-03-16':'2018-03-31',j].to_frame()
        stepwise_model = auto_arima(train_df, start_p=0, start_q=0,
                           max_p=2, max_q=2, m=7,max_P=2,max_Q=2,
                           start_Q=0,start_P=0,seasonal=True,
                           d=1, D=1, trace=False,
                           error_action='ignore',
                           suppress_warnings=True,
                           stepwise=True)
        stepwise_model.fit(test_df)
        predicted_val = stepwise_model.predict(n_periods=16)
        test_pred.append(predicted_val)
        pred = pd.DataFrame(predicted_val,index=test_df.index,columns=[j])
        print(pred)
        plt.figure()
        plt.plot(pd.concat([train_df,test_df],axis=1),label='orignal')
        plt.plot(pred,label='pred')
        plt.legend()
        mape.append(mean_absolute_percentage_error(test_df[j],pred[j]))
        smape_error.append(smape(test_df[j].tolist(),test_pred[i]))
    #pd.DataFrame(smape_error).T.to_excel('smape_female.xlsx')
    print(smape_error)
prediction_mape(outlier_free_df_male)    

ValueError: maxlag should be < nobs

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

def ets_prediction(df):
    test_pred = []
    mape = []
    smape_error = []
    i=0
    for j in df.columns:

        train_ets = df.loc['2018-01-01':'2018-03-15',j].to_frame()
        test_ets = df.loc['2018-03-16':'2018-03-31',j].to_frame()

        ETS = ExponentialSmoothing(train_ets,trend='add',damped=True).fit(smoothing_level=0.2,smoothing_slope=0.1,optimized=True)
        predicted_val = ETS.predict(start='2018-03-16',end='2018-03-31')
        test_pred.append(predicted_val)
        pred = pd.DataFrame(predicted_val,index=test_ets.index,columns=[j])
        plt.figure()
        plt.plot(pd.concat([train_ets,test_ets],axis=1),label='orignal')
        plt.plot(pred,label='pred')
        plt.legend()
        mape.append(mean_absolute_percentage_error(test_ets[j],pred[j]))
        smape_error.append(smape(test_ets[j].tolist(),test_pred[i]))
        i = i+1
    pd.DataFrame(smape_error).T.to_excel('ets_other_smape.xlsx') 

ets_prediction(outlier_free_df_other)


In [ ]:
def forecast_arima(df):
    forecast_pred = []
    mape = []
    i=0
    smape_error = []
    for j in df.columns:
        stepwise_model = auto_arima(df[j], start_p=0, start_q=0,
                           max_p=2, max_q=2, m=1,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=False,
                           error_action='ignore',
                           suppress_warnings=True, 
                           stepwise=True,stationary=True)
        stepwise_model.fit(df[j])
        predicted_val = stepwise_model.predict(n_periods=15)
        forecast_pred.append(predicted_val)
    forecast_df = pd.DataFrame(forecast_pred)
    forecast_t = forecast_df.T
    forecast_t.to_excel('forecast_arima_other.xlsx')

forecast_arima(outlier_free_df_other)    

In [ ]:
def ets_forecast(df):
    forecast_pred = []
    i=0
    for j in df.columns:
        ETS = ExponentialSmoothing(df[j],trend='add',damped=True).fit(smoothing_level=0.2,smoothing_slope=0.1,optimized=True)
        predicted_val = ETS.predict(start='2018-04-01',end='2018-04-15')
        forecast_pred.append(predicted_val)
    forecast_df = pd.DataFrame(forecast_pred)
    forecast_t = forecast_df.T
    forecast_t.to_excel('forecast_ets_other.xlsx')

ets_forecast(outlier_free_df_other)


In [ ]:
train_df_ageband1 = age_band1_final.loc['2018-01-01':'2018-03-15','title985'].to_frame()
test_df_ageband1  = age_band1_final.loc['2018-03-16':'2018-03-31','title985'].to_frame()
# train_df_ageband1.columns = ['title985']
# test_df_ageband1.columns = ['title985']

In [ ]:
model = ARIMA(train_df_ageband1,order=(2,0,2))
result_ARMA = model.fit(disp=-1)

# plt.Figure(figsize=(15,8))
# plt.plot(train_df_ageband1,color='red',label='outlierfree')
# # plt.plot(dframe_agegroup1,color='blue',label='orignal')
# plt.plot(result_ARMA.fittedvalues,color='black',label='predicted')
# plt.legend()
# result_ARMA.predict(periods=15)
pred = result_ARMA.predict(start='2018-03-16',end='2018-03-31')
plt.plot(pd.concat([train_df_ageband1,pred],axis=0))
plt.plot(age_band1_final['title985'])
mean_absolute_percentage_error(test_df_ageband1['title985'],pred)

In [ ]:
from pyramid.arima import auto_arima
stepwise_model = auto_arima(train_df_ageband1, start_p=0, start_q=0,
                       max_p=2, max_q=2, m=1,
                       start_P=0, seasonal=True,
                       d=1, D=1, trace=True,
                       error_action='ignore',
                       suppress_warnings=True, 
                       stepwise=True)
#stepwise_model.fit(test_df_ageband1)
pred = pd.DataFrame(stepwise_model.predict(n_periods=16),index=test_df_ageband1.index,columns=['title985'])
pred
plt.plot(pd.concat([train_df_ageband1,pred],axis=1))
# plt.plot(test_df_ageband1)
plt.plot(age_band1_final[985.0])
mean_absolute_percentage_error(test_df_ageband1['title985'],pred['title985'])

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [ ]:
# q[985.0].plot()
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X,y)
# y_pred = regressor.predict(test_df.iloc[:,0:6])
# import pyflux as pf
# model = pf.ARIMAX(data=q, formula='impressions~age+gender',
#                   ar=1, ma=1, family=pf.Normal())
# x = model.fit("MLE")
# x.summary()

# # model.predict(h=20,oos_data=train_date_agg.iloc[-12:],intervals=True)
# model.predict(h=10,oos_data=train_date_agg.iloc[-10:],intervals=True)

In [ ]:
train_df = df_ageband1[df_ageband1['date'].between('2018-01-01','2018-03-15')]
train_df.head()

In [ ]:
test_df = df_ageband1[df_ageband1['date'].between('2018-03-16','2018-03-31')]
test_df.head()

In [ ]:
train_df = train_df[train_df.impressions != 0]
test_df = test_df[test_df.impressions != 0]

In [ ]:
X = train_df.iloc[:,0:7]
y = train_df.iloc[:,7]